In [1]:
# Do a pseudo code for vectorize photos of zeros and ones
# use SVD and PCA
# do the algebra logic by "hand", dont use scikit-learn


In [ ]:
# Importamos las librerías necesarias
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:


# Función para cargar, procesar y vectorizar imágenes
def cargar_procesar_imagenes(directorio, nuevo_tam=(50, 50)):
    imagenes_vectorizadas = []
    
    for archivo in os.listdir(directorio):
        if archivo.endswith(".jpeg"):
            # Cargar imagen en escala de grises
            ruta_imagen = os.path.join(directorio, archivo)
            img = cv2.imread(ruta_imagen, cv2.IMREAD_GRAYSCALE)
            if img is not None:
                # Reducir el tamaño de la imagen
                img_redimensionada = cv2.resize(img, nuevo_tam)
                # Aumentar contraste usando equalización de histograma
                img_contraste = cv2.equalizeHist(img_redimensionada)
                # Vectorizar la imagen (convertir la matriz en un vector)
                imagenes_vectorizadas.append(img_contraste.flatten())
    
    return np.array(imagenes_vectorizadas)

# 1. Cargar y procesar las imágenes del directorio './fotos/'
directorio = "./fotos/"
X = cargar_procesar_imagenes(directorio)

# Verificamos que se hayan cargado imágenes
if X.shape[0] == 0:
    print("No se encontraron imágenes .jpeg en el directorio especificado.")
else:
    print(f"Se han cargado {X.shape[0]} imágenes con {X.shape[1]} características cada una.")

# 2. Estandarización: centramos los datos (restamos la media de cada característica)
X_mean = np.mean(X, axis=0)
X_centered = X - X_mean / 255.0

# 3. No usar la matriz de covarianza, sino la matriz de productos internos
XtX = X_centered.T @ X_centered

# 4. Calcular los vectores y valores propios de la matriz de productos internos
eigvals, eigvecs = np.linalg.eig(XtX)


# 5. Ordenar los valores propios y vectores en orden descendente
indices_orden = np.argsort(eigvals)[::-1]
eigvals = eigvals[indices_orden]
eigvecs = eigvecs[:, indices_orden]

# 6. Seleccionar los 2 primeros componentes principales para visualización
PCs = eigvecs[:, :2]

# 7. Proyección de los datos en el espacio de los componentes principales
X_pca = X_centered @ PCs

# 8. Clasificación simple: '1' si PC1 > media, '0' en caso contrario
umbral = np.mean(X_pca[:, 0])
etiquetas = (X_pca[:, 0] > umbral).astype(int)

# 8. Visualización: scatter plot de las proyecciones PCA
plt.figure(figsize=(8,6))
scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=etiquetas, cmap='bwr', alpha=0.7)
plt.xlabel("Primer Componente Principal (PC1)")
plt.ylabel("Segundo Componente Principal (PC2)")
plt.title("Distribución de imágenes en el espacio PCA\n(rojo=1, azul=0)")
plt.colorbar(scatter, label="Etiqueta")
plt.grid(True)
plt.show()

# 9. Mostrar la cantidad de imágenes clasificadas como '1' y '0'
num_1 = np.sum(etiquetas)
num_0 = len(etiquetas) - num_1
print(f"Imágenes clasificadas como '1': {num_1}")
print(f"Imágenes clasificadas como '0': {num_0}")

# %% [markdown]
# ## Introducción y Teoría
# 
# **Objetivo**: Clasificar imágenes de dígitos '0' y '1' mediante técnicas de álgebra lineal (SVD y PCA), reduciendo la dimensionalidad de los datos y visualizando patrones.
# 
# ### Fundamentos Matemáticos
# 1. **Vectorización de Imágenes**:
#   - Una imagen en escala de grises se representa como matriz 2D de pixeles (valores 0-255).
#   - Para análisis numérico, se "aplana" a un vector 1D de tamaño `ancho × alto`.
# 
# 2. **Descomposición en Valores Singulares (SVD)**:
#   - Cualquier matriz \( X \) puede descomponerse como:
#     \[
#     X = U \Sigma V^T
#     \]
#     donde \( V^T \) contiene los vectores singulares (direcciones de máxima varianza).
# 
# 3. **Análisis de Componentes Principales (PCA)**:
#   - Método estadístico para proyectar datos en direcciones ortogonales de máxima varianza.
#   - Los componentes principales son los autovectores de la matriz de covarianza \( X^TX \).
# 
# ### Relación SVD-PCA
# - Las columnas de \( V \) en SVD corresponden a los componentes principales.
# - Los valores singulares \( \Sigma \) se relacionan con la varianza explicada: \( \lambda_i = \sigma_i^2/(n-1) \).

# %%
# Importación de librerías
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2

# %% [markdown]
# ## Desarrollo y Pormenores (Procesamiento de Imágenes)
# 
# ### Etapa 1: Preprocesamiento
# **Propósito**: Normalizar características para análisis efectivo.
# 1. **Escala de Grises**: Elimina información redundante de color.
# 2. **Reducción de Tamaño (75x75)**:
#   - Reduce dimensionalidad computacional.
#   - Elimina ruido y bordes irrelevantes.
# 3. **Aumento de Contraste**:
#   - Equalización de histograma redistribuye intensidades de pixeles.
#   - Mejora la separabilidad entre dígitos y fondo.

# %%
def cargar_procesar_imagenes(directorio, nuevo_tam=(50, 50)):
    imagenes_vectorizadas = []
    for archivo in os.listdir(directorio):
        if archivo.endswith(".jpeg"):
            img = cv2.imread(os.path.join(directorio, archivo), cv2.IMREAD_GRAYSCALE)
            if img is not None:
                img_redim = cv2.resize(img, nuevo_tam)  # Uniformiza dimensiones
                img_eq = cv2.equalizeHist(img_redim)     # Mejora contraste
                imagenes_vectorizadas.append(img_eq.flatten())  # Vectorización
    return np.array(imagenes_vectorizadas)

# %% [markdown]
# ### Etapa 2: Estandarización de Datos
# **Por qué centrar?**:
# - PCA es sensible a la media de los datos.
# - Restar la media garantiza que el primer componente principal capture la dirección de máxima varianza verdadera, no artefactos por desplazamiento.

# %%
X_mean = np.mean(X, axis=0)        # Calcula media por pixel
X_centered = X - X_mean/255.0      # Centrado escalado (evita overflow numérico)

# %% [markdown]
# ### Etapa 3: Cálculo de Componentes Principales
# **Matriz de Covarianza** \( X^TX \):
# - Captura correlaciones entre pixeles.
# - Autovectores = Direcciones principales de variación.
# - Autovalores = Magnitud de varianza en cada dirección.

# %%
XtX = X_centered.T @ X_centered           # Matriz de covarianza (producto interno)
eigvals, eigvecs = np.linalg.eig(XtX)     # Descomposición espectral

# Orden descendente por varianza explicada
indices_orden = np.argsort(eigvals)[::-1]
eigvals = eigvals[indices_orden]          # Autovalores ordenados
eigvecs = eigvecs[:, indices_orden]       # Autovectores ordenados

# %% [markdown]
# ## Resultados y Visualización
# 
# ### Proyección en 2D
# - **PC1 y PC2** capturan las direcciones de máxima y segunda máxima varianza.
# - La separación en el gráfico sugiere que PC1 diferencia entre clases.

# %%
PCs = eigvecs[:, :2]                # Selecciona 2 primeros componentes
X_pca = X_centered @ PCs            # Proyección lineal

plt.scatter(X_pca[:,0], X_pca[:,1], c=etiquetas, cmap='bwr')
plt.xlabel('PC1 (Varianza: {:.1f}%)'.format(100*eigvals[0]/np.sum(eigvals)))
plt.ylabel('PC2 (Varianza: {:.1f}%)'.format(100*eigvals[1]/np.sum(eigvals)))

# %% [markdown]
# ## Conclusiones
# 
# 1. **Efectividad de PCA**:
#   - Reducción de 5,625 características (75x75) a solo 2 manteniendo estructura.
#   - Separación visual sugiere que los dígitos tienen patrones espaciales distintivos.
# 
# 2. **Limitaciones**:
#   - Clasificación simple por umbral podría fallar con datos no lineales.
#   - Sensible a rotaciones o traslaciones en imágenes.
# 
# 3. **Extensiones Futuras**:
#   - Usar SVD truncado para compresión antes de PCA.
#   - Implementar clasificador SVM en el espacio PCA.
# 
# **Aplicaciones**: Reconocimiento de caracteres manuscritos, detección de anomalías en imágenes médicas.

# %%
# Resultados de clasificación
print(f"Clasificados como '1': {num_1} | Clasificados como '0': {num_0}")

% \newpage

% \vfill
\bibliographystyle{siam}
\bibliography{referencias}

\begin{thebibliography}{1}
\bibitem{Golub} Golub, G. H., \& Van Loan, C. F. (2013). Matrix computations (Vol. 3). JHU Press.
\bibitem{McKinney} **McKinney, W.** (2017). *Python for Data Analysis* (2nd ed.). O’Reilly Media. ISBN: 978-1491957660

\end{thebibliography}

ejemplo de referencia: \cite{   } y \cite{  }.

¿Para qué sirve \vfill? Esto se utiliza para llenar el espacio restante de la página con espacios en blanco. Se puede utilizar para centrar contenido 

# Si aparece dos veces el titulo "referencias" se puede solucionar con el siguiente comando 
\renewcommand{\refname}{}
\renewcommand{\refname}{Referencias}



(Due to technical issues, the search service is temporarily unavailable.)

Here’s a **bibliography** tailored to the theoretical foundations of SVD, PCA, and image analysis discussed in your report. These references include textbooks, research papers, and authoritative resources for mathematical and applied contexts:

---

### **Books**
1. **Strang, G.** (2006). *Linear Algebra and Its Applications* (4th ed.).  
   - *Relevance*: Classic textbook explaining the theory of Singular Value Decomposition (SVD) and its geometric interpretation (Chapter 6).  
   - ISBN: 978-0030105678

2. **Jolliffe, I. T.** (2002). *Principal Component Analysis* (2nd ed.). Springer.  
   - *Relevance*: Definitive guide to PCA, including proofs, applications, and connections to covariance matrices.  
   - ISBN: 978-0387954424

3. **Bishop, C. M.** (2006). *Pattern Recognition and Machine Learning*. Springer.  
   - *Relevance*: Chapter 12 covers PCA in the context of dimensionality reduction and latent variable models.  
   - ISBN: 978-0387310732

4. **Golub, G. H., & Van Loan, C. F.** (2013). *Matrix Computations* (4th ed.). Johns Hopkins University Press.  
   - *Relevance*: Detailed algorithms for SVD and eigenvalue decomposition (Chapters 2 and 8).  
   - ISBN: 978-1421407944

---

### **Research Papers**
5. **Turk, M., & Pentland, A.** (1991). *Eigenfaces for Recognition*. Journal of Cognitive Neuroscience.  
   - *Relevance*: Seminal paper applying PCA (called eigenfaces) to facial recognition.  
   - DOI: [10.1162/jocn.1991.3.1.71](https://doi.org/10.1162/jocn.1991.3.1.71)

6. **Shlens, J.** (2014). *A Tutorial on Principal Component Analysis*. arXiv.  
   - *Relevance*: Intuitive derivation of PCA with geometric interpretations.  
   - arXiv: [1404.1100](https://arxiv.org/abs/1404.1100)

---

### **Applied Resources**
7. **McKinney, W.** (2017). *Python for Data Analysis* (2nd ed.). O’Reilly Media.  
   - *Relevance*: Practical implementation of PCA/SVD using NumPy and SciPy (Chapter 13).  
   - ISBN: 978-1491957660

8. **VanderPlas, J.** (2016). *Python Data Science Handbook*. O’Reilly Media.  
   - *Relevance*: Hands-on examples of PCA for image compression and visualization.  
   - Free access: [Python Data Science Handbook](https://jakevdp.github.io/PythonDataScienceHandbook/)

9. **OpenCV Documentation** (2023). *Image Processing and Computer Vision*.  
   - *Relevance*: Official docs for image preprocessing steps (e.g., histogram equalization, resizing).  
   - Link: [OpenCV Documentation](https://docs.opencv.org/)

---

### **Online Tutorials and Tools**
10. **NumPy Documentation** (2023). *numpy.linalg.svd*.  
    - *Relevance*: Official guide to SVD implementation in NumPy.  
    - Link: [numpy.linalg.svd](https://numpy.org/doc/stable/reference/generated/numpy.linalg.svd.html)

11. **Scikit-learn Documentation** (2023). *Decomposing signals in components (matrix factorization problems)*.  
    - *Relevance*: Theory and code for PCA, though your report avoids using scikit-learn.  
    - Link: [Scikit-learn Decomposition](https://scikit-learn.org/stable/modules/decomposition.html)

---

### **Specialized Topics**
12. **Goodfellow, I., Bengio, Y., & Courville, A.** (2016). *Deep Learning*. MIT Press.  
    - *Relevance*: Chapter 2 (Linear Algebra) and Chapter 5 (Machine Learning Basics) contextualize PCA in modern ML.  
    - Free access: [Deep Learning Book](https://www.deeplearningbook.org/)

---

### **How to Cite**
For academic reports, use APA/MLA/IEEE formats depending on requirements. Example (APA):  
> Strang, G. (2006). *Linear Algebra and Its Applications* (4th ed.). Brooks Cole.

Let me know if you need help formatting citations for a specific style! 📚